In [1]:
from vectordb_module import FAISSDatabaseManager
import pandas as pd
train_df = pd.read_csv('train.csv')
# DB 생성 및 검색 예시
train_db_manager = FAISSDatabaseManager(db_path='./serm_best_field_train_db', chunk_strategy="serm", search_strategy="serm_best_field")
train_db_manager.make_db(train_df)

test_df = pd.read_csv('test.csv')
test_db_manager = FAISSDatabaseManager(db_path='./serm_best_field_test_db', chunk_strategy="serm", search_strategy="serm_best_field")
test_db_manager.make_db(train_df)

# Fewshot DB 생성
fewshot_manager = FAISSDatabaseManager(db_path='./fewshot_db',search_strategy="knn_best_field")
fewshot_manager.make_db(train_df,fewshot=True)

c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Loading FAISS DB from: ./serm_best_field_train_db
FAISS DB and documents loaded from ./serm_best_field_train_db
Loading FAISS DB from: ./serm_best_field_test_db
FAISS DB and documents loaded from ./serm_best_field_test_db


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


FAISS DB and documents saved to ./fewshot_db


In [2]:
print('Test 1')
query = train_df.iloc[309]['Question']
print('Query:', query)
# 쿼리를 기반으로 검색 수행
results = test_db_manager.search_documents(query, top_k=3)
# 결과 출력
for i, result in enumerate(results):
    print(f"Result {i+1}: {result.page_content[:200]}...")

Test 1
Query: 외국인 배우자 수급권 부여는 언제부터 시행되었는가?
FAISS DB and documents loaded from ./serm_best_field_test_db
Result 1: - 2 -5.사업근거 및 추진경위 ①법령상 근거 및 조항 적시 :아동수당법 제4조제 5항 ◈아동수당법제4조(’23.9.14.시행) 제4조(아동수당의지급대상및지급액)①아동수당은8세미만의아동에게매월10만원을지급한다. ⑤제1항에도불구하고만2세미만아동에게매월50만원이상으로서대통령령으로정 하는금액을추가로지급한다. 부칙제6조(아동수당추가지급금액에관한특례)제4조제5항의...
Result 2: - 2 --(기타 의료급여수급권자 급여비용 국가부담금 )「국민기초생활 보장법」에 의한 의료 급여수급권자를 제외한 기타 *의료급여수급권자의 장기요양급여 이용에 따른 급여 비용 및 관리운영비 국고지원 (서울 50%,기타지역 80%) *이재민,의사상자,국가유공자,입양아동,국가무형문화재보유자,북한이탈주민등 -(노인장기요양보험 사업관리 )노인장기요양보험 사업추진에 ...
Result 3: 09 최소운영수입보장(BTO 등) ‣ 최소운영수입보장 제도란 주로 BTO 민간투자사업 중 실시협약서 상 추정 수입보다 실제 수입이 미치지 못하는 경우 정부가 최소운영수입을 보장해 주는 제도를 말함16) - 국토교통부, 해양수산부의 민간투자사업 중 협약 상 최소운영수입을 보장하는 사업은 2020회계연도 말 기준 16건 존재하며, 협약 내용은 <표9>와 같음 ...


In [5]:
print('Test 1')
query =  test_df.iloc[78]['Question']
print('Query:', query)
# 쿼리를 기반으로 검색 수행
results = test_db_manager.search_documents(query, top_k=3)
# 결과 출력
for i, result in enumerate(results):
    print(f"Result {i+1}: {result.metadata}...")
    print(f"Result {i+1}: {result.page_content[:200]}...")

Test 1
Query: 우리나라에서는 언제부터 발생주의 기준을 적용한 국가결산보고서에서 우발부채를 보고하고 있는가?
FAISS DB and documents loaded from ./serm_best_field_test_db
Result 1: {'source': './test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf', 'page': 1}...
Result 1: 적용한 국가결산보고서가 10년간 작성되었으나 우발부채의 인식과 공시는 크게 달라지지 않음(현재 주석에서 ‘우발사항 및 약정사항’으로 보고) - 우리나라는 국가회계기준, 지방회계기준, 한국채택국제회계기준(K–IFRS) 등에서 우발부채를 광의적 개념에 서만 정의하고 있어 용어 사용 혼란 및 체계적 관리 한계(본고 4페이지 참고) - SNA, PSDS, IPSA...
Result 2: {'source': './test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf', 'page': 1}...
Result 2: 02 FIS ISSUE & FOCUS 들어가며 ISSUE 왜 우발부채에 주목하는가? ‣ 국제기준 재정통계 산출에 발생주의(accrual basis)1) 회계기준이 적용되면서 미래의 다양한 의제의무 (constructive obligation)2)를 포괄하는 우발부채 관리에 대한 중요성 인식 - 기업회계에서는 우발부채 규모가 그리 크지 않아 중요한 요소가 아...
Result 3: {'source': './test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf', 'page': 4}...
Result 3: 「2020회계연도 국가결산보고서」를 참고하여 재작성. 10) 국제회계기준(International Finncial Reporting Standards, IFRS)은 전세계의 민간기업들의 회계작성법을 통일된 기준으로 만든 것, 국제공공부문회계기준 (International Public Sector Ac

In [4]:
query = test_df.iloc[5]['Question']
results = fewshot_manager.search_documents(query, top_k=3)

print('Test 2')

print('Query:', query)
# 결과 출력
for i, result in enumerate(results):
    if isinstance(result, dict):  # Few-shot 방식의 결과
        print(f"Result {i+1}: {result}...")
    else:  # 일반 문서 검색 결과
        print(f"Result {i+1}: {result.page_content[:200]}...")

FAISS DB and documents loaded from ./fewshot_db
Test 2
Query: 재창업자금이 재도약지원자금으로 이관된 연도는 언제인가요?
Result 1: {'Source': '2024년도 성과계획서(총괄편)', 'Source_path': './train_source/2024년도 성과계획서(총괄편).pdf', 'Question': '국가재정법 시행으로 성과관리제도가 법제화된 연도는 언제인가요?', 'Answer': '2007년'}...
Result 2: {'Source': '중소벤처기업부_창업사업화지원', 'Source_path': './train_source/중소벤처기업부_창업사업화지원.pdf', 'Question': "언제 청년 일자리 대책의 일환으로 기술혁신 기반 예비창업자의 사업화를 지원하는 '기술혁신형 창업기업 지원사업'이 신설되었나요?", 'Answer': "기술혁신형 창업기업 지원사업(현, 예비창업패키지)'은 '18년에 신설되었습니다."}...
Result 3: {'Source': '중소벤처기업부_창업사업화지원', 'Source_path': './train_source/중소벤처기업부_창업사업화지원.pdf', 'Question': '창업사업화지원 사업의 법령상 근거는 무엇인가요?', 'Answer': '「중소기업창업 지원법」 제10조(창업 활성화 지원사업의 추진 등)과 「기술의 이전 및 사업화 촉진에 관한 법률」 제15조(기술이전ㆍ사업화 촉진사업의 추진)'}...
